In [3]:
import numpy as np
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import nltk
import pandas as pd

AttributeError: type object 'pandas._libs.lib.Seen' has no attribute '__reduce_cython__'

In [ ]:
data = pd.read_csv(r"./phone_data.csv",encoding  ='UTF-8')
data = data.fillna(method = 'ffill')
data.head(20)

In [ ]:

words = list(set(data['vals'].values))
tags = list(set(data['keys'].values))
len(words)

In [ ]:
class Getter(object):
    def __init__(self,data):
        self.data = data
        func = lambda x :[(w,t) for w,t in zip(x['vals'].values.tolist(),
                                                   x['keys'].values.tolist())]
        self.grouped = self.data.groupby('grp').apply(func)
        self.sentences = [s for s in self.grouped]
        


In [ ]:
getter = Getter(data)
sentences = getter.sentences
print(sentences[1])

In [ ]:
word2idx = {w:i for i,w in enumerate(words)}
tags2idx = {t:i for i,t in enumerate(tags)}
word2idx


In [ ]:
tags2idx

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
for s in sentences :
    for w in s:
        if w[0] in stop_words :
            s.pop(s.index(w))
   

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len =15
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen = max_len,sequences = X,padding = 'post',value = len(words)-1)

Y = [[tags2idx[t[-1]] for t in s] for s in sentences]
Y = pad_sequences(maxlen = max_len,sequences = Y,padding = 'post',value = tags2idx['O'])
Y = [ to_categorical(i,num_classes = len(tags)) for i in Y]
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.1,random_state=1)
Y_train


In [ ]:
from tensorflow.keras.layers import Embedding,Dense,Dropout,LSTM,Bidirectional,TimeDistributed
from tensorflow.keras import Model,Input
input_word = Input(shape=(max_len,))
model = Embedding(input_dim = len(words),input_length = max_len,output_dim= max_len)(input_word)
model = Dropout(0.2)(model)
model = Bidirectional(LSTM(units = 30,return_sequences = True,recurrent_dropout = 0.1))(model)
model =  TimeDistributed(Dense(len(tags),activation = 'relu'))(model)
out = TimeDistributed(Dense(len(tags),activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

In [13]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.005)
model.compile(optimizer = adam,loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [14]:

history = model.fit(X_train,np.array(Y_train),validation_split = 0.2,batch_size = 16,epochs = 24,verbose = 1)

Train on 271 samples, validate on 68 samples
Epoch 1/24
271/271 [==============================] - 6s 23ms/sample - loss: 1.5033 - accuracy: 0.6770 - val_loss: 0.7498 - val_accuracy: 0.7412
Epoch 2/24
271/271 [==============================] - 1s 2ms/sample - loss: 0.6900 - accuracy: 0.7459 - val_loss: 0.5696 - val_accuracy: 0.8176
Epoch 3/24
271/271 [==============================] - 1s 2ms/sample - loss: 0.5286 - accuracy: 0.7702 - val_loss: 0.4875 - val_accuracy: 0.7824
Epoch 4/24
271/271 [==============================] - 1s 2ms/sample - loss: 0.4528 - accuracy: 0.7833 - val_loss: 0.4245 - val_accuracy: 0.8206
Epoch 5/24
271/271 [==============================] - 0s 2ms/sample - loss: 0.3887 - accuracy: 0.8403 - val_loss: 0.3854 - val_accuracy: 0.8373
Epoch 6/24
271/271 [==============================] - 0s 2ms/sample - loss: 0.3484 - accuracy: 0.8499 - val_loss: 0.3654 - val_accuracy: 0.8412
Epoch 7/24
271/271 [==============================] - 0s 2ms/sample - loss: 0.3111 - accur

In [15]:
model.evaluate(X_test,np.array(Y_test))

38/38 [==============================] - 0s 267us/sample - loss: 0.4230 - accuracy: 0.9018


[0.42298581568818344, 0.9017544]

In [16]:
i = np.random.randint(0,X_test.shape[0])
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p,axis = -1)
print("Word" + "              " + "Tag")
for w,t,pred in zip(X_test[i],Y_test[i],p[0]):
    print(str(words[w]) + "              " + str(tags[pred]))

Word              Tag
aaxa              Brand
projector              Category
rechargeable              Category
aa              O
battery              Category
ab              ModelName
switch              ModelName
ahs              O
ahs              O
ahs              O
ahs              O
ahs              O
ahs              O
ahs              O
ahs              O
